In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
pd_ori=pd.read_csv('all.csv')

In [53]:
pd_ori.head()

,Unnamed: 0,user_id,product_id,label,rating,date_review,review
0,0,5044,0,1,1.0,2014-11-16,"Drinks were bad, the hot chocolate was watered..."
1,1,5045,0,1,1.0,2014-09-08,This was the worst experience I've ever had a ...
2,2,5046,0,1,3.0,2013-10-06,This is located on the site of the old Spruce ...
3,3,5047,0,1,5.0,2014-11-30,I enjoyed coffee and breakfast twice at Toast ...
4,4,5048,0,1,5.0,2014-08-28,I love Toast! The food choices are fantastic -...


In [3]:
review=pd_ori['review']

In [4]:
nltk_tokenizer = RegexpTokenizer(r'\w+')
def tokenizer(text):
    return nltk_tokenizer.tokenize(text)

In [5]:
review_length=[]
for i in range(len(review)):
    review_length.append(len(tokenizer(review[i])))

In [6]:
review_date=pd_ori['date_review']
review_date_copy=review_date.copy()

In [7]:
for i in range(len(review_date)):
    review_date_copy[i]=int(review_date[i].replace('-',''))

In [8]:
review_date_copy

0         20141116
1         20140908
2         20131006
3         20141130
4         20140828
            ...   
608593    20130120
608594    20121112
608595    20120822
608596    20110511
608597    20100717
Name: date_review, Length: 608598, dtype: object

In [9]:
review_rating=pd_ori['rating']

In [10]:
reviewer_id=pd_ori['user_id']

In [11]:
restaurant_id=pd_ori['product_id']

In [12]:
metadata_restaurant=pd.DataFrame({"review_length":review_length,
                                  'review_date':review_date_copy,
                                  'reviewer':reviewer_id,
                                  'restaurant':restaurant_id,
                                   'review_rating': review_rating,
                                  'label':pd_ori['label']
                                 })

In [13]:
type(metadata_restaurant['review_date'][0])

int

In [46]:
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(metadata_restaurant, metadata_restaurant['label']):
    strat_train_set=metadata_restaurant.iloc[train_index]
    strat_test_set=metadata_restaurant.iloc[test_index]
strat_test_set['label'].value_counts()/len(strat_test_set)

0    0.867787
1    0.132213
Name: label, dtype: float64

In [15]:
X_train = strat_train_set[[
    'review_length', 'review_date', 'reviewer', 'restaurant', 'review_rating'
]]

In [16]:
X_train

,review_length,review_date,reviewer,restaurant,review_rating
349337,105,20130311,7155,3044,4.0
448380,110,20100822,15868,3806,3.0
229140,144,20130221,74732,1905,2.0
471511,82,20100107,109028,3899,5.0
292393,83,20140106,99863,1250,5.0
...,...,...,...,...,...
460939,0,20131109,59926,3875,4.0
392385,93,20101221,12658,3314,5.0
189210,24,20130930,118784,56,4.0
573720,93,20141126,17220,4757,2.0


In [17]:
y_train=strat_train_set[['label']]


In [18]:
X_test = strat_test_set[[
    'review_length', 'review_date', 'reviewer', 'restaurant', 'review_rating'
]]

In [19]:
def result_evalation(predict_result, true_result):
    print("accuracy:", accuracy_score(true_result, predict_result))
    print("precision:", precision_score(true_result, predict_result))
    print("recall:", recall_score(true_result, predict_result))
    print("f1:", f1_score(true_result, predict_result))


In [20]:
y_test=strat_test_set[['label']]


In [21]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train,y_train['label'].tolist())
y_pred=model.predict(X_test)

In [22]:
result_evalation(y_pred,y_test)

accuracy: 0.8667762076897798
precision: 0.3237822349570201
recall: 0.007021686447523768
f1: 0.013745286461501032


In [23]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_pred, y_test['label'].tolist()))

0.5960599057392972


In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
model=LogisticRegression(solver='liblinear', multi_class='auto')
model.fit(X_train,y_train['label'].tolist())
y_pred=model.predict(X_test)

In [26]:
result_evalation(y_pred,y_test)

accuracy: 0.8677867236279987
precision: 0.0
recall: 0.0
f1: 0.0


/Users/sunqi/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print(roc_auc_score(y_pred, y_test['label'].tolist()))

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [47]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)
for train_index, test_index in sss.split(strat_test_set, strat_test_set['label']):
    strat_train_set=strat_test_set.iloc[train_index]
    strat_test_set=strat_test_set.iloc[test_index]
strat_test_set['label'].value_counts()/len(strat_test_set)

0    0.867791
1    0.132209
Name: label, dtype: float64

In [48]:
len(strat_train_set)

24344

In [49]:
X_train = strat_train_set[[
    'review_length', 'review_date', 'reviewer', 'restaurant', 'review_rating'
]]
X_test = strat_test_set[[
    'review_length', 'review_date', 'reviewer', 'restaurant', 'review_rating'
]]
y_test=strat_test_set[['label']]
y_train=strat_train_set[['label']]

In [50]:
len(X_train)

24344

In [51]:
model=SVC(gamma='scale',kernel='rbf', probability=True)
model.fit(X_train,y_train['label'].tolist())
y_pred=model.predict(X_test)

In [52]:
print(roc_auc_score(y_pred, y_test['label'].tolist()))

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.